# CUDA setup is NOT needed if using GPU runtime but needs to install pyCUDA

- pycuda: Python CUDA interface
- pycublas: Additional Python bindings to simplify matrix multiplication operations
- cupy: CUDA numpy replacement

In [1]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
! pip install pycuda

     |████████████████████████████████| 1.6MB 8.2MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620705 sha256=b3b12effbdfe2cffd790195a569f7ad91eefeb09c6bcea901a205585f0135c48
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.4-py2.py3-none-any.whl size=59109 sha256=65390662f9d8d9732f724599ea40aa1c6372b16bde158061e66bc41e21b08ed5
  Stored in directory: /root/.cache/pip/wheels/6f/2a/6e/0b210d900f2b6caca133395d2fa153d8a9c03ad42c3c5d55bf
Successfully built pycuda pytools


In [14]:
! pip install pycublas

ERROR: Could not find a version that satisfies the requirement pycublas (from versions: none)
ERROR: No matching distribution found for pycublas


In [3]:
import numpy as np
import pandas as pd

import pycuda
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [4]:
# -- initialize the device
import pycuda.autoinit

### Get GPU info

In [5]:
def gpu_memstat():
    (free, total) = cuda.mem_get_info()
    print("Global memory occupancy:\n%f total\n%f%% free" % (free * 100 / total, total))    

In [6]:
for devicenum in range(cuda.Device.count()):
    device=cuda.Device(devicenum)
    attrs=device.get_attributes()

    #Beyond this point is just pretty printing
    print("\n===Attributes for device %d"%devicenum)
    for key, value in attrs.items():
        print("%s:%s"%(str(key),str(value)))


===Attributes for device 0
ASYNC_ENGINE_COUNT:3
CAN_MAP_HOST_MEMORY:1
CLOCK_RATE:1590000
COMPUTE_CAPABILITY_MAJOR:7
COMPUTE_CAPABILITY_MINOR:5
COMPUTE_MODE:DEFAULT
CONCURRENT_KERNELS:1
ECC_ENABLED:1
GLOBAL_L1_CACHE_SUPPORTED:1
GLOBAL_MEMORY_BUS_WIDTH:256
GPU_OVERLAP:1
INTEGRATED:0
KERNEL_EXEC_TIMEOUT:0
L2_CACHE_SIZE:4194304
LOCAL_L1_CACHE_SUPPORTED:1
MANAGED_MEMORY:1
MAXIMUM_SURFACE1D_LAYERED_LAYERS:2048
MAXIMUM_SURFACE1D_LAYERED_WIDTH:32768
MAXIMUM_SURFACE1D_WIDTH:32768
MAXIMUM_SURFACE2D_HEIGHT:65536
MAXIMUM_SURFACE2D_LAYERED_HEIGHT:32768
MAXIMUM_SURFACE2D_LAYERED_LAYERS:2048
MAXIMUM_SURFACE2D_LAYERED_WIDTH:32768
MAXIMUM_SURFACE2D_WIDTH:131072
MAXIMUM_SURFACE3D_DEPTH:16384
MAXIMUM_SURFACE3D_HEIGHT:16384
MAXIMUM_SURFACE3D_WIDTH:16384
MAXIMUM_SURFACECUBEMAP_LAYERED_LAYERS:2046
MAXIMUM_SURFACECUBEMAP_LAYERED_WIDTH:32768
MAXIMUM_SURFACECUBEMAP_WIDTH:32768
MAXIMUM_TEXTURE1D_LAYERED_LAYERS:2048
MAXIMUM_TEXTURE1D_LAYERED_WIDTH:32768
MAXIMUM_TEXTURE1D_LINEAR_WIDTH:134217728
MAXIMUM_TEXTURE1D

### Default CUDA kernel block dim.
- Number of Threads per block (or block size), can NOT be greater than **1024**. 
- device Attribute `MAX_THREADS_PER_BLOCK`
- ```block size = dim_x * dim_y * dim_z```

In [7]:
default_block_dim = (512, 1, 1)

### Doublify inplace

In [11]:
doublify_inplace_mod = SourceModule("""
  __global__ void doublify_inplace(float *a, int n)
  {
    int i = threadIdx.x + blockDim.x * blockIdx.x;                                                                                                                                                    
    
    if (i < n)
        a[i] *= 2;
  }
  """)

doublify_inplace_cuda = doublify_inplace_mod.get_function("doublify_inplace")

def doublify_inplace(a, block_dim=default_block_dim):
    # 1. Allocate device memory
    a_gpu = cuda.mem_alloc(a.nbytes)
    # 2. Copy data from host to device
    cuda.memcpy_htod(a_gpu, a)
    # 3. Run Kernel (block size will match a's shape)
    
    num_blocks = int(np.ceil(a.size / block_dim[0]))
    grid_dim = (num_blocks, 1)

    doublify_inplace_cuda(a_gpu, np.int32(a.size), block=block_dim, grid=grid_dim)
    # 4. Retrieve results from device
    a_doubled = np.empty_like(a)
    cuda.memcpy_dtoh(a_doubled, a_gpu)
                                                    4ereturn a_doubled

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 2))



CompileError: ignored

### Doublify

In [ ]:
doublify_mod = SourceModule("""
  __global__ void doublify(float *result, float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    result[idx] = a[idx] * 2;
  }
  """)

doublify_cuda = doublify_mod.get_function("doublify")

def doublify(a):
    # Allocate host result variable
    a_doubled = np.zeros_like(a)
    # Run Kernel (block size will match a's shape)
    doublify_cuda(cuda.Out(a_doubled), cuda.In(a), block=(*a.shape, 1))
    return a_doubled






### Test

In [62]:
shape = (320, 320)
a = np.random.randn(*shape).astype(np.float32)

In [63]:
a.shape, a.size

((320, 320), 102400)

In [52]:
a_doubled1 = doublify_inplace(a)
# a_doubled2 = doublify(a)

In [53]:
print(a)
print()
print(a_doubled1)
print()
print(2*a - a_doubled1)

[[-1.0675634  -0.6881886   1.3184122  ...  1.6380174   2.495172
   0.34937605]
 [-1.500824    0.50660264 -1.0992174  ... -0.06016374 -0.65936977
  -0.1762956 ]
 [-0.09020101 -0.2336011   0.8268078  ... -0.34031394  0.14124267
   0.06431733]
 ...
 [ 0.6436912  -0.18807298 -1.1498893  ... -0.77654284  0.8835537
  -0.618134  ]
 [-0.48769403  1.5980868   1.795433   ... -0.3136548  -1.326746
  -0.9133636 ]
 [ 1.0415727   1.042258    0.8173271  ... -0.03603094 -0.48960242
  -0.40469465]]

[[-2.1351268  -1.3763772   2.6368244  ...  3.2760348   4.990344
   0.6987521 ]
 [-3.001648    1.0132053  -2.1984348  ... -0.12032747 -1.3187395
  -0.3525912 ]
 [-0.18040203 -0.4672022   1.6536156  ... -0.6806279   0.28248534
   0.12863466]
 ...
 [ 1.2873824  -0.37614596 -2.2997787  ... -1.5530857   1.7671074
  -1.236268  ]
 [-0.97538805  3.1961737   3.590866   ... -0.6273096  -2.653492
  -1.8267272 ]
 [ 2.0831454   2.084516    1.6346542  ... -0.07206188 -0.97920483
  -0.8093893 ]]

[[0. 0. 0. ... 0. 0. 0.]


### Benchmark / Profiling

In [ ]:
shape = (32000, 32000)
a = np.random.randn(*shape).astype(np.float32)

In [114]:
shape[0] * shape[1]

102400000

In [115]:
%timeit -n 10 d = doublify_inplace(a, (512, 1, 1))

10 loops, best of 3: 186 ms per loop


In [102]:
gpu_memstat()

Global memory occupancy:99.258111% free


In [116]:
%timeit -n 10 d = 2 * a

10 loops, best of 3: 71.5 ms per loop


# Simpler example (not too great)

In [ ]:
# Kernel
matrix_multiply_cuda = SourceModule("""
__global__ void matrix_multiply(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

matrix_multiply = matrix_multiply_cuda.get_function("matrix_multiply")

In [ ]:
n = 1000

In [ ]:
def create_matrices(n):
    a = np.random.randn(n).astype(np.float32)
    b = np.random.randn(n).astype(np.float32)
    return a, b

In [ ]:
%%timeit -n 10000

a, b = create_matrices(n)
dest = np.zeros_like(a)

matrix_multiply(cuda.Out(dest), cuda.In(a), cuda.In(b), block=(n, 1, 1))

10000 loops, best of 3: 336 µs per loop


In [ ]:
%%timeit -n 10000

a, b = create_matrices(n)

dest2 = a * b

10000 loops, best of 3: 77.9 µs per loop


# SAXPY

In [ ]:
saxpy_cuda = comp.SourceModule("""
/**
 * Kernel: Single-precision a*x plus y
 * In CUDA, kernels are defined using the __global__ declaration specifier.
 * the __device__ specifier can be used to declare functions used only by kernel functions.
 */
__global__ void saxpy(int n, float a, float *x, float *y)
{
	// blockDim: contains the dimensions of each thread block as specified in the second execution configuration parameter for the kernel launch.
	// blockIdx: index of the thread block within the grid
	// threadIdx: index of the thread within its thread block
	int i = blockIdx.x * blockDim.x + threadIdx.x;

	// This check is required for cases where the number of elements in an array is not evenly divisible by the thread block size,
	// and as a result the number of threads launched by the kernel is larger than the array size.
	if (i < n) y[i] = a * x[i] + y[i];
}
""")

saxpy = saxpy_cuda.get_function("saxpy")

# Just a reference on how to setup CUDA

In [ ]:
!curl https://colab.chainer.org/install

#!/bin/sh

#
# Chainer/CuPy Installer for Google Colaboratory
# https://github.com/chainer/google-colaboratory
#


if ! nvidia-smi > /dev/null; then
    cat << '_EOF_'
********************************************************************************
GPU is not enabled!
Open "Runtime" > "Change runtime type" and set "Hardware accelerator" to "GPU".
********************************************************************************
_EOF_
    exit 1
fi


if [ -d /usr/local/cuda-10.0 ]; then
    # For CUDA 10.0 container
    set -ex
    apt -y -q install cuda-libraries-dev-10-0
    pip install -q "cupy-cuda100 ${CUPY_VERSION}" "chainer ${CHAINER_VERSION}"

elif [ -d /usr/local/cuda-9.2 ]; then
    # For CUDA 9.2 container
    set -ex
    apt -y -q install cuda-libraries-dev-9-2
    pip install -q "cupy-cuda92 ${CUPY_VERSION}" "chainer ${CHAINER_VERSION}"

elif [ -e /usr/lib/x86_64-linux-gnu/libcudart.so.8.0 ]; then
    # For CUDA 8.0 container
    set -ex
    apt -y -q install libcusparse8.0 l